## Import library

In [1]:
import os
import pickle

import numpy as np
import seaborn as sns
import pandas as pd
import scipy
from matplotlib import pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential, layers, callbacks
from tensorflow.keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional
from tensorflow.keras.optimizers import RMSprop, Adam

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from IPython.core.pylabtools import figsize

## Load data

In [2]:
# for natural sorting
import re

def tryint(s):
    try:
        return int(s)
    except:
        return s
def alphanum_key(s):
    """ Turn a string into a list of string and number chunks.
        "z23a" -> ["z", 23, "a"]
    """
    return [ tryint(c) for c in re.split('([0-9]+)', s) ]
def sort_nicely(l):
    """ Sort the given list in the way that humans expect.
    """
    l.sort(key=alphanum_key)
    

# load features data
dir_path = "/tmp/elias/emotion_recognition/features"

dir_list = os.listdir(dir_path)
dir_list.sort()
dir_list.pop(0)
sort_nicely(dir_list)

path = "/tmp/elias/emotion_recognition/features/"
features_data = []
count = 0
for file in dir_list:
    if count == 2:
        break
    data = np.loadtxt(path+file, delimiter=',',skiprows=1, usecols=range(1,19))
    features_data.append(data)
    count = count + 1

print("length of features data :", len(features_data))    


# load labels data    
dir_list_ = os.listdir("/tmp/elias/emotion_recognition/data")
dir_list_.sort()
dir_list_ = dir_list_[:21]

labels_Valence = []
labels_Arousal = []

count = 0
for file in dir_list_:
    if count == 2:
        break
    dat_file = '/tmp/elias/emotion_recognition/data/' + file
    with open(dat_file, 'rb') as f:
        Channel_data =pickle.load(f,encoding='latin1')
    labels = Channel_data["labels"]
    for value in labels:
        if value[0] < 5:
            labels_Valence.append(0)
        if value[0] >= 5:
            labels_Valence.append(1)
        if value[1] < 5:
            labels_Arousal.append(0)
        if value[1] >= 5:
            labels_Arousal.append(1)
    count = count + 1

print("length of Arousal labels data :", len(labels_Arousal), "length of Valence labels data :", len(labels_Valence))

length of features data : 2
length of Arousal labels data : 80 length of Valence labels data : 80


In [3]:
# make sure you loaded data with no error
print(dir_list[0])
print(features_data[0])
print(labels_Arousal[0])
print(labels_Valence[0])

0vid_1_1.csv
[[104.91666667 106.5          0.         ...   0.         100.
    0.        ]
 [104.91666667 106.5          0.         ...   0.         100.
    0.        ]
 [104.91666667 106.5          0.         ...   0.         100.
    0.        ]
 ...
 [105.90909091 114.           1.         ...   0.         100.
    0.        ]
 [105.90909091 114.           1.         ...   0.         100.
    0.        ]
 [105.90909091 114.           1.         ...   0.         100.
    0.        ]]
1
1


## Assign GPU memory

In [4]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 2 GB of memory on the first GPU
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4000)]) # limit in megabytes
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


2022-02-06 15:15:16.433177: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2022-02-06 15:15:16.480998: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1555] Found device 0 with properties: 
pciBusID: 0000:02:00.0 name: NVIDIA GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.545GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2022-02-06 15:15:16.481286: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2022-02-06 15:15:16.483887: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2022-02-06 15:15:16.486144: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10
2022-02-06 15:15:16.486538: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcu

## Split data into train and test
- what we have to do for this
 - 동영상 기준? ㅁ
 - 샘플 기준?

In [5]:
# train 80%, test 20%
train_data = features_data[:2]
test_data = features_data[672:]
train_data_Vlabels = labels_Valence[:2]
train_data_Alabels = labels_Arousal[:2]
test_data_Vlabels = labels_Valence[672:]
test_data_Alabels = labels_Arousal[672:]
print(len(train_data))
print(len(test_data))

2
0


## Set train and test data

In [6]:
x_train_data = np.array(train_data)
x_test_data = np.array(test_data)
y_train_data = np.array(train_data_Vlabels)
x_train_data.shape , y_train_data.shape
print(x_train_data)

# normalize data
nsamples, nx, ny = x_train_data.shape
train_dataset = x_train_data.reshape((nsamples,nx*ny))
scaler = MinMaxScaler(feature_range=(0, 1))
x_train_data = scaler.fit_transform(train_dataset)

###
# X_train = x_train_data.reshape((x_train_data.shape[0], x_train_data.shape[1], 18))
X_train = x_train_data.reshape((x_train_data.shape[0], 6400, 18))
Y_train = y_train_data.reshape((y_train_data.shape[0], 1))
# array([[a],[b],[c]...])

X_train.shape

[[[104.91666667 106.5          0.         ...   0.         100.
     0.        ]
  [104.91666667 106.5          0.         ...   0.         100.
     0.        ]
  [104.91666667 106.5          0.         ...   0.         100.
     0.        ]
  ...
  [105.90909091 114.           1.         ...   0.         100.
     0.        ]
  [105.90909091 114.           1.         ...   0.         100.
     0.        ]
  [105.90909091 114.           1.         ...   0.         100.
     0.        ]]

 [[102.58333333 111.5          0.         ...   0.         100.
     0.        ]
  [102.58333333 111.5          0.         ...   0.         100.
     0.        ]
  [102.58333333 111.5          0.         ...   0.         100.
     0.        ]
  ...
  [109.27272727 109.           0.         ...   0.         100.
     0.        ]
  [109.27272727 109.           0.         ...   0.         100.
     0.        ]
  [109.27272727 109.           0.         ...   0.         100.
     0.        ]]]


(2, 6400, 18)

## Build RNN, LSTM, GRU model

In [7]:
# Set Hyperparameters of the model
learning_rate = 0.001
epochs = 100
batch_size=64

In [8]:
X_train[0].shape

(6400, 18)

In [9]:
model1 = Sequential()
model1.add(GRU(units=64, activation='relu', input_shape=(6400,18), return_sequences=True))
model1.add(GRU(units=128, return_sequences=True))
# model1.add(GRU(units=256, return_sequences=True))
model1.add(GRU(units=256))

## gotta add dropout!
# model.add(GRU(units=128, return_sequences=True, return_state=True,dropout=0.2))

# Dense net
model1.add(Dense(256))
model1.add(Dense(128))
model1.add(Dense(64))
model1.add(Dense(1))

model1.compile(loss='binary_crossentropy',optimizer=Adam(lr=learning_rate),metrics=['mae'])
# model1.compile(loss='binary_crossentropy',optimizer=Adam(lr=learning_rate),metrics=['accuracy'])
model1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru (GRU)                    (None, 6400, 64)          16128     
_________________________________________________________________
gru_1 (GRU)                  (None, 6400, 128)         74496     
_________________________________________________________________
gru_2 (GRU)                  (None, 256)               296448    
_________________________________________________________________
dense (Dense)                (None, 256)               65792     
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 6

In [10]:
hist=model1.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, verbose=1)

Train on 2 samples
Epoch 1/100


2022-02-06 15:15:20.391492: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2022-02-06 15:15:22.304784: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7


2/2 [==============================] - 15s 7s/sample - loss: 9.1292 - mae: 0.9827
Epoch 2/100
2/2 [==============================] - 11s 5s/sample - loss: 0.0000e+00 - mae: 8.6050
Epoch 3/100
2/2 [==============================] - 11s 5s/sample - loss: 0.0000e+00 - mae: 17.5170
Epoch 4/100
2/2 [==============================] - 11s 6s/sample - loss: 0.0000e+00 - mae: 25.0699
Epoch 5/100
2/2 [==============================] - 11s 6s/sample


KeyboardInterrupt: 

In [ ]:
model3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
hist=model3.fit(X_train,Y_train,epochs=epochs,batch_size=batch_size)

In [ ]:
# plot